In [ ]:
import numpy as np
import torch
import torch_geometric as pyg
import graphPINN
import math
import logging
from time import time
from tqdm.notebook import tqdm
from scipy.io import savemat, loadmat
import os

In [ ]:
folder = "/home/nasa/code/checkpoints/2023-03-30/"
if not os.path.exists(f'{folder}extraps/'):
    os.makedirs(f'{folder}extraps/')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
k = 100

dataset = graphPINN.data.MHSDataset(f'/mnt/d/data_k={k}',k=k)


#trainset, validset, testset = torch.utils.data.random_split(dataset,[0.8, 0.1, 0.1],generator=torch.Generator().manual_seed(314))
trainset, validset, testset = torch.utils.data.random_split(dataset,[0.005, 0.001, 0.994],generator=torch.Generator().manual_seed(314))

model = torch.load(f'{folder}model_trainsize-18_k-100_params-90.pt').to(device)

In [ ]:
set = 'train'

for j in tqdm(range(5)):
    if set == 'train':
        data = trainset[j]
    elif set == 'validate':
        data = validset[j]
    else: # set == 'test'
        data = testset[j]

    data = data.to_homogeneous().to(device)
    pred = model(data)[0].detach()

    mat = {'Bml':pred[0].cpu().numpy(),
           'Btn':data.y.cpu().numpy(),
           'nodes':data.pos.cpu().numpy(),
           'pres':data.x[:,3:6].detach().cpu().numpy()}
    savemat(f'{folder}/extraps/{set}{j}.mat',mat)